In [2]:
import os
import time
import numpy as np
import pandas as pd
import scipy.io as sio
from IPython.display import display
import itertools as it

import matplotlib.pyplot as plt
import pywt
import scipy.stats

import datetime as dt
from collections import defaultdict, Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold 

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

In [3]:
#Face Touching datasets

data1fta=pd.read_csv('Acc FT 1.0.csv')
data2fta=pd.read_csv('Acc FT 2.0.csv')
data3fta=pd.read_csv('Acc FT 3.0.csv')
data4fta=pd.read_csv('Acc FT 4.0.csv')
data5fta=pd.read_csv('Acc FT 5.0.csv')
data6fta=pd.read_csv('Acc FT 6.0.csv')

data1ftg=pd.read_csv('Gyro FT 1.0.csv')
data2ftg=pd.read_csv('Gyro FT 2.0.csv')
data3ftg=pd.read_csv('Gyro FT 3.0.csv')
data4ftg=pd.read_csv('Gyro FT 4.0.csv')
data5ftg=pd.read_csv('Gyro FT 5.0.csv')
data6ftg=pd.read_csv('Gyro FT 6.0.csv')


#Mask wearing-removing datasets

data1mwra=pd.read_csv('Acc MWR 1.0.csv')
data2mwra=pd.read_csv('Acc MWR 2.0.csv')
data3mwra=pd.read_csv('Acc MWR 3.0.csv')
data4mwra=pd.read_csv('Acc MWR 4.0.csv')
data5mwra=pd.read_csv('Acc MWR 5.0.csv')
data6mwra=pd.read_csv('Acc MWR 6.0.csv')

data1mwrg=pd.read_csv('Gyro MWR 1.0.csv')
data2mwrg=pd.read_csv('Gyro MWR 2.0.csv')
data3mwrg=pd.read_csv('Gyro MWR 3.0.csv')
data4mwrg=pd.read_csv('Gyro MWR 4.0.csv')
data5mwrg=pd.read_csv('Gyro MWR 5.0.csv')
data6mwrg=pd.read_csv('Gyro MWR 6.0.csv')

In [4]:
data_colsfta = ['time', 'accelx', 'accely','accelz', 'absolute_accel']
data1fta.columns = data_colsfta
data2fta.columns = data_colsfta
data3fta.columns = data_colsfta
data4fta.columns = data_colsfta
data5fta.columns = data_colsfta
data6fta.columns = data_colsfta


data_colsftg = ['time', 'gyrox', 'gyroy','gyroz', 'absolute_gyro']
data1ftg.columns = data_colsftg
data2ftg.columns = data_colsftg
data3ftg.columns = data_colsftg
data4ftg.columns = data_colsftg
data5ftg.columns = data_colsftg
data6ftg.columns = data_colsftg


data_colsmwra = ['time', 'accelx', 'accely','accelz', 'absolute_accel']
data1mwra.columns = data_colsmwra
data2mwra.columns = data_colsmwra
data3mwra.columns = data_colsmwra
data4mwra.columns = data_colsmwra
data5mwra.columns = data_colsmwra
data6mwra.columns = data_colsmwra

data_colsmwrg = ['time', 'gyrox', 'gyroy','gyroz', 'absolute_gyro']
data1mwrg.columns = data_colsmwrg
data2mwrg.columns = data_colsmwrg
data3mwrg.columns = data_colsmwrg
data4mwrg.columns = data_colsmwrg
data5mwrg.columns = data_colsmwrg
data6mwrg.columns = data_colsmwrg

In [5]:
data1mwra.shape

(8182, 5)

In [6]:
data1mwrg.shape

(8151, 5)

In [7]:
def moving_window(acceld, length, step=1):
    streams = it.tee(acceld, length)
    return zip(*[it.islice(stream, i, None, step*length) for stream, i in zip(streams, it.count(step=step))])


# len(accelx1_) =129 = length in overlap

def overlap(acceld_,length):
    newd=[]
    wd=[]
    for i in range ((length-1)):
        newd=np.concatenate((acceld_[i], acceld_[i+1]))        
        i=i+1
        wd.append(newd)
        
    return wd

def final_window(step):
#     step=stepd.accelx
    step_= list(moving_window(step, 64))
    step_=np.asarray(step_)
    l = len(step_)
    wstep=[]
    wstep=overlap(step_,l)
    wstep=np.asarray(wstep)
    return wstep

In [8]:
## FaceTouching accelerometer

#### x axis
w1ftax=[]
w1ftax=final_window(data1fta.accelx)

w2ftax=[]
w2ftax=final_window(data2fta.accelx)

w3ftax=[]
w3ftax=final_window(data3fta.accelx)

w4ftax=[]
w4ftax=final_window(data4fta.accelx)

w5ftax=[]
w5ftax=final_window(data5fta.accelx)

w6ftax=[]
w6ftax=final_window(data6fta.accelx)

w1ftax=np.asarray(w1ftax)
w2ftax=np.asarray(w2ftax)
w3ftax=np.asarray(w3ftax)
w4ftax=np.asarray(w4ftax)
w5ftax=np.asarray(w5ftax)
w6ftax=np.asarray(w6ftax)

wftax = []
wftax=np.concatenate((w1ftax,w2ftax))
wftax=np.concatenate((wftax, w3ftax))
wftax=np.concatenate((wftax, w4ftax))
wftax=np.concatenate((wftax, w5ftax))
wftax=np.concatenate((wftax, w6ftax))

#### y axis
w1ftay=[]
w1ftay=final_window(data1fta.accely)

w2ftay=[]
w2ftay=final_window(data2fta.accely)

w3ftay=[]
w3ftay=final_window(data3fta.accely)

w4ftay=[]
w4ftay=final_window(data4fta.accely)

w5ftay=[]
w5ftay=final_window(data5fta.accely)

w6ftay=[]
w6ftay=final_window(data6fta.accely)

w1ftay=np.asarray(w1ftay)
w2ftay=np.asarray(w2ftay)
w3ftay=np.asarray(w3ftay)
w4ftay=np.asarray(w4ftay)
w5ftay=np.asarray(w5ftay)
w6ftay=np.asarray(w6ftay)

wftay = []
wftay=np.concatenate((w1ftay,w2ftay))
wftay=np.concatenate((wftay, w3ftay))
wftay=np.concatenate((wftay, w4ftay))
wftay=np.concatenate((wftay, w5ftay))
wftay=np.concatenate((wftay, w6ftay))

#### z axis
w1ftaz=[]
w1ftaz=final_window(data1fta.accelz)

w2ftaz=[]
w2ftaz=final_window(data2fta.accelz)

w3ftaz=[]
w3ftaz=final_window(data3fta.accelz)

w4ftaz=[]
w4ftaz=final_window(data4fta.accelz)

w5ftaz=[]
w5ftaz=final_window(data5fta.accelz)

w6ftaz=[]
w6ftaz=final_window(data6fta.accelz)

w1ftaz=np.asarray(w1ftaz)
w2ftaz=np.asarray(w2ftaz)
w3ftaz=np.asarray(w3ftaz)
w4ftaz=np.asarray(w4ftaz)
w5ftaz=np.asarray(w5ftaz)
w6ftaz=np.asarray(w6ftaz)

wftaz = []
wftaz=np.concatenate((w1ftaz,w2ftaz))
wftaz=np.concatenate((wftaz, w3ftaz))
wftaz=np.concatenate((wftaz, w4ftaz))
wftaz=np.concatenate((wftaz, w5ftaz))
wftaz=np.concatenate((wftaz, w6ftaz))

In [9]:
wftax.shape

(768, 128)

In [10]:
wftay.shape

(768, 128)

In [11]:
wftaz.shape

(768, 128)

In [13]:
## FaceTouching gyroscope

#### x axis
w1ftgx=[]
w1ftgx=final_window(data1ftg.gyrox)

w2ftgx=[]
w2ftgx=final_window(data2ftg.gyrox)

w3ftgx=[]
w3ftgx=final_window(data3ftg.gyrox)

w4ftgx=[]
w4ftgx=final_window(data4ftg.gyrox)

w5ftgx=[]
w5ftgx=final_window(data5ftg.gyrox)

w6ftgx=[]
w6ftgx=final_window(data6ftg.gyrox)

w1ftgx=np.asarray(w1ftgx)
w2ftgx=np.asarray(w2ftgx)
w3ftgx=np.asarray(w3ftgx)
w4ftgx=np.asarray(w4ftgx)
w5ftgx=np.asarray(w5ftgx)
w6ftgx=np.asarray(w6ftgx)

wftgx = []
wftgx=np.concatenate((w1ftgx,w2ftgx))
wftgx=np.concatenate((wftgx, w3ftgx))
wftgx=np.concatenate((wftgx, w4ftgx))
wftgx=np.concatenate((wftgx, w5ftgx))
wftgx=np.concatenate((wftgx, w6ftgx))

#### y axis
w1ftgy=[]
w1ftgy=final_window(data1ftg.gyroy)

w2ftgy=[]
w2ftgy=final_window(data2ftg.gyroy)

w3ftgy=[]
w3ftgy=final_window(data3ftg.gyroy)

w4ftgy=[]
w4ftgy=final_window(data4ftg.gyroy)

w5ftgy=[]
w5ftgy=final_window(data5ftg.gyroy)

w6ftgy=[]
w6ftgy=final_window(data6ftg.gyroy)

w1ftgy=np.asarray(w1ftgy)
w2ftgy=np.asarray(w2ftgy)
w3ftgy=np.asarray(w3ftgy)
w4ftgy=np.asarray(w4ftgy)
w5ftgy=np.asarray(w5ftgy)
w6ftgy=np.asarray(w6ftgy)

wftgy = []
wftgy=np.concatenate((w1ftgy,w2ftgy))
wftgy=np.concatenate((wftgy, w3ftgy))
wftgy=np.concatenate((wftgy, w4ftgy))
wftgy=np.concatenate((wftgy, w5ftgy))
wftgy=np.concatenate((wftgy, w6ftgy))

#### z axis
w1ftgz=[]
w1ftgz=final_window(data1ftg.gyroz)

w2ftgz=[]
w2ftgz=final_window(data2ftg.gyroz)

w3ftgz=[]
w3ftgz=final_window(data3ftg.gyroz)

w4ftgz=[]
w4ftgz=final_window(data4ftg.gyroz)

w5ftgz=[]
w5ftgz=final_window(data5ftg.gyroz)

w6ftgz=[]
w6ftgz=final_window(data6ftg.gyroz)

w1ftgz=np.asarray(w1ftgz)
w2ftgz=np.asarray(w2ftgz)
w3ftgz=np.asarray(w3ftgz)
w4ftgz=np.asarray(w4ftgz)
w5ftgz=np.asarray(w5ftgz)
w6ftgz=np.asarray(w6ftgz)

wftgz = []
wftgz=np.concatenate((w1ftgz,w2ftgz))
wftgz=np.concatenate((wftgz, w3ftgz))
wftgz=np.concatenate((wftgz, w4ftgz))
wftgz=np.concatenate((wftgz, w5ftgz))
wftgz=np.concatenate((wftgz, w6ftgz))

In [14]:
wftgx.shape

(756, 128)

In [15]:
wftgy.shape

(756, 128)

In [16]:
wftgz.shape

(756, 128)

In [28]:
## MaskWearingRemoving accelerometer

#### x axis
w1mwrax=[]
w1mwrax=final_window(data1mwra.accelx)

w2mwrax=[]
w2mwrax=final_window(data2mwra.accelx)

w3mwrax=[]
w3mwrax=final_window(data3mwra.accelx)

w4mwrax=[]
w4mwrax=final_window(data4mwra.accelx)

w5mwrax=[]
w5mwrax=final_window(data5mwra.accelx)

w6mwrax=[]
w6mwrax=final_window(data6mwra.accelx)

w1mwrax=np.asarray(w1mwrax)
w2mwrax=np.asarray(w2mwrax)
w3mwrax=np.asarray(w3mwrax)
w4mwrax=np.asarray(w4mwrax)
w5mwrax=np.asarray(w5mwrax)
w6mwrax=np.asarray(w6mwrax)

wmwrax = []
wmwrax=np.concatenate((w1mwrax,w2mwrax))
wmwrax=np.concatenate((wmwrax, w3mwrax))
wmwrax=np.concatenate((wmwrax, w4mwrax))
wmwrax=np.concatenate((wmwrax, w5mwrax))
wmwrax=np.concatenate((wmwrax, w6mwrax))

#### y axis
w1mwray=[]
w1mwray=final_window(data1mwra.accely)

w2mwray=[]
w2mwray=final_window(data2mwra.accely)

w3mwray=[]
w3mwray=final_window(data3mwra.accely)

w4mwray=[]
w4mwray=final_window(data4mwra.accely)

w5mwray=[]
w5mwray=final_window(data5mwra.accely)

w6mwray=[]
w6mwray=final_window(data6mwra.accely)

w1mwray=np.asarray(w1mwray)
w2mwray=np.asarray(w2mwray)
w3mwray=np.asarray(w3mwray)
w4mwray=np.asarray(w4mwray)
w5mwray=np.asarray(w5mwray)
w6mwray=np.asarray(w6mwray)

wmwray = []
wmwray=np.concatenate((w1mwray,w2mwray))
wmwray=np.concatenate((wmwray, w3mwray))
wmwray=np.concatenate((wmwray, w4mwray))
wmwray=np.concatenate((wmwray, w5mwray))
wmwray=np.concatenate((wmwray, w6mwray))

#### z axis
w1mwraz=[]
w1mwraz=final_window(data1mwra.accelz)

w2mwraz=[]
w2mwraz=final_window(data2mwra.accelz)

w3mwraz=[]
w3mwraz=final_window(data3mwra.accelz)

w4mwraz=[]
w4mwraz=final_window(data4mwra.accelz)

w5mwraz=[]
w5mwraz=final_window(data5mwra.accelz)

w6mwraz=[]
w6mwraz=final_window(data6mwra.accelz)

w1mwraz=np.asarray(w1mwraz)
w2mwraz=np.asarray(w2mwraz)
w3mwraz=np.asarray(w3mwraz)
w4mwraz=np.asarray(w4mwraz)
w5mwraz=np.asarray(w5mwraz)
w6mwraz=np.asarray(w6mwraz)

wmwraz = []
wmwraz=np.concatenate((w1mwraz,w2mwraz))
wmwraz=np.concatenate((wmwraz, w3mwraz))
wmwraz=np.concatenate((wmwraz, w4mwraz))
wmwraz=np.concatenate((wmwraz, w5mwraz))
wmwraz=np.concatenate((wmwraz, w6mwraz))

In [29]:
wmwrax.shape

(766, 128)

In [30]:
wmwray.shape

(766, 128)

In [31]:
wmwraz.shape

(766, 128)

In [35]:
## MaskWearingRemoving gyroscope

#### x axis
w1mwrgx=[]
w1mwrgx=final_window(data1mwrg.gyrox)

w2mwrgx=[]
w2mwrgx=final_window(data2mwrg.gyrox)

w3mwrgx=[]
w3mwrgx=final_window(data3mwrg.gyrox)

w4mwrgx=[]
w4mwrgx=final_window(data4mwrg.gyrox)

w5mwrgx=[]
w5mwrgx=final_window(data5mwrg.gyrox)

w6mwrgx=[]
w6mwrgx=final_window(data6mwrg.gyrox)

w1mwrgx=np.asarray(w1mwrgx)
w2mwrgx=np.asarray(w2mwrgx)
w3mwrgx=np.asarray(w3mwrgx)
w4mwrgx=np.asarray(w4mwrgx)
w5mwrgx=np.asarray(w5mwrgx)
w6mwrgx=np.asarray(w6mwrgx)

wmwrgx = []
wmwrgx=np.concatenate((w1mwrgx,w2mwrgx))
wmwrgx=np.concatenate((wmwrgx, w3mwrgx))
wmwrgx=np.concatenate((wmwrgx, w4mwrgx))
wmwrgx=np.concatenate((wmwrgx, w5mwrgx))
wmwrgx=np.concatenate((wmwrgx, w6mwrgx))

#### y axis
w1mwrgy=[]
w1mwrgy=final_window(data1mwrg.gyroy)

w2mwrgy=[]
w2mwrgy=final_window(data2mwrg.gyroy)

w3mwrgy=[]
w3mwrgy=final_window(data3mwrg.gyroy)

w4mwrgy=[]
w4mwrgy=final_window(data4mwrg.gyroy)

w5mwrgy=[]
w5mwrgy=final_window(data5mwrg.gyroy)

w6mwrgy=[]
w6mwrgy=final_window(data6mwrg.gyroy)

w1mwrgy=np.asarray(w1mwrgy)
w2mwrgy=np.asarray(w2mwrgy)
w3mwrgy=np.asarray(w3mwrgy)
w4mwrgy=np.asarray(w4mwrgy)
w5mwrgy=np.asarray(w5mwrgy)
w6mwrgy=np.asarray(w6mwrgy)

wmwrgy = []
wmwrgy=np.concatenate((w1mwrgy,w2mwrgy))
wmwrgy=np.concatenate((wmwrgy, w3mwrgy))
wmwrgy=np.concatenate((wmwrgy, w4mwrgy))
wmwrgy=np.concatenate((wmwrgy, w5mwrgy))
wmwrgy=np.concatenate((wmwrgy, w6mwrgy))

#### z axis
w1mwrgz=[]
w1mwrgz=final_window(data1mwrg.gyroz)

w2mwrgz=[]
w2mwrgz=final_window(data2mwrg.gyroz)

w3mwrgz=[]
w3mwrgz=final_window(data3mwrg.gyroz)

w4mwrgz=[]
w4mwrgz=final_window(data4mwrg.gyroz)

w5mwrgz=[]
w5mwrgz=final_window(data5mwrg.gyroz)

w6mwrgz=[]
w6mwrgz=final_window(data6mwrg.gyroz)

w1mwrgz=np.asarray(w1mwrgz)
w2mwrgz=np.asarray(w2mwrgz)
w3mwrgz=np.asarray(w3mwrgz)
w4mwrgz=np.asarray(w4mwrgz)
w5mwrgz=np.asarray(w5mwrgz)
w6mwrgz=np.asarray(w6mwrgz)

wmwrgz = []
wmwrgz=np.concatenate((w1mwrgz,w2mwrgz))
wmwrgz=np.concatenate((wmwrgz, w3mwrgz))
wmwrgz=np.concatenate((wmwrgz, w4mwrgz))
wmwrgz=np.concatenate((wmwrgz, w5mwrgz))
wmwrgz=np.concatenate((wmwrgz, w6mwrgz))

In [36]:
wmwrgx.shape

(756, 128)

In [37]:
wmwrgy.shape

(756, 128)

In [38]:
wmwrgz.shape

(756, 128)

In [39]:
activities_description = {
    1: 'Face Touching',
    2: 'Mask Removing and Wearing'
}

In [40]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

def randomize_data(dataset):
    permutation = np.random.permutation(dataset.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    return shuffled_dataset

In [44]:
wftax_r=[]
wftax_r = wftax[0:341]
wftax_r=np.concatenate((wftax_r,wftax[346:490]))
wftax_r=np.concatenate((wftax_r,wftax[495:766]))
wftax_r.shape

(756, 128)

In [45]:
wftay_r=[]
wftay_r = wftay[0:341]
wftay_r=np.concatenate((wftay_r,wftay[346:490]))
wftay_r=np.concatenate((wftay_r,wftay[495:766]))
wftay_r.shape

(756, 128)

In [46]:
wftaz_r=[]
wftaz_r = wftaz[0:341]
wftaz_r=np.concatenate((wftaz_r,wftaz[346:490]))
wftaz_r=np.concatenate((wftaz_r,wftaz[495:766]))
wftaz_r.shape

(756, 128)

In [48]:
ft=np.stack((wftax_r,wftay_r,wftaz_r,wftgx,wftgy,wftgz))
ft.shape

(6, 756, 128)

In [76]:
ft=ft.transpose(1,2,0)
ft.shape

(756, 128, 6)

In [77]:
wmwrax_r=[]
wmwrax_r = wmwrax[0:340]
wmwrax_r=np.concatenate((wmwrax_r,wmwrax[342:490]))
wmwrax_r=np.concatenate((wmwrax_r,wmwrax[498:768]))
wmwrax_r.shape

(756, 128)

In [78]:
wmwray_r=[]
wmwray_r = wmwray[0:340]
wmwray_r=np.concatenate((wmwray_r,wmwray[342:490]))
wmwray_r=np.concatenate((wmwray_r,wmwray[498:768]))
wmwray_r.shape

(756, 128)

In [79]:
wmwraz_r=[]
wmwraz_r = wmwraz[0:340]
wmwraz_r=np.concatenate((wmwraz_r,wmwraz[342:490]))
wmwraz_r=np.concatenate((wmwraz_r,wmwraz[498:768]))
wmwraz_r.shape

(756, 128)

In [80]:
mwr=np.stack((wmwrax_r,wmwray_r,wmwraz_r,wmwrgx,wmwrgy,wmwrgz))
mwr.shape

(6, 756, 128)

In [81]:
mwr=mwr.transpose(1,2,0)
mwr.shape

(756, 128, 6)

In [82]:
mwr= randomize_data(mwr)
mwr.shape

ft= randomize_data(ft)
ft.shape

(756, 128, 6)

In [83]:
# 70% of 756 = 529.2
x_trainft=ft[:529]
x_testft= ft[529:]

x_trainmwr=mwr[:529]
x_testmwr= mwr[529:]

In [84]:
x_testft.shape

(227, 128, 6)

In [85]:
x_train=[]
x_train=np.concatenate((x_trainft, x_trainmwr))
x_train.shape

(1058, 128, 6)

In [86]:
x_test=[]
x_test=np.concatenate((x_testft, x_testmwr))
x_test.shape

(454, 128, 6)

In [87]:
y_trainft=np.full((529), 1, dtype=int)
y_trainmwr=np.full((529), 2, dtype=int)


y_testft=np.full((227), 1, dtype=int)
y_testmwr=np.full((227), 2, dtype=int)


In [88]:
y_train=[]
y_train=np.concatenate((y_trainft, y_trainmwr))
y_train.shape

(1058,)

In [89]:
y_test=[]
y_test=np.concatenate((y_testft, y_testmwr))
y_test.shape

(454,)

In [90]:
waveletname = 'db8'
skf = StratifiedKFold(n_splits=10, random_state=None)
# X is the feature set and y is the target
for train_index, test_index in skf.split(x_train,y_train): 
    print("Train:", train_index, "Validation:", test_index) 
    signals_train, signals_test = x_train[train_index], x_train[test_index] 
    labels_train, labels_test = y_train[train_index], y_train[test_index]    
    X_train, Y_train = get_uci_har_features(signals_train, labels_train, waveletname)
    X_test, Y_test = get_uci_har_features(signals_test, labels_test, waveletname)
    models = batch_classify(X_train, Y_train, X_test, Y_test)
    display_dict_models(models)

Train: [  53   54   55   56   57   58   59   60   61   62   63   64   65   66
   67   68   69   70   71   72   73   74   75   76   77   78   79   80
   81   82   83   84   85   86   87   88   89   90   91   92   93   94
   95   96   97   98   99  100  101  102  103  104  105  106  107  108
  109  110  111  112  113  114  115  116  117  118  119  120  121  122
  123  124  125  126  127  128  129  130  131  132  133  134  135  136
  137  138  139  140  141  142  143  144  145  146  147  148  149  150
  151  152  153  154  155  156  157  158  159  160  161  162  163  164
  165  166  167  168  169  170  171  172  173  174  175  176  177  178
  179  180  181  182  183  184  185  186  187  188  189  190  191  192
  193  194  195  196  197  198  199  200  201  202  203  204  205  206
  207  208  209  210  211  212  213  214  215  216  217  218  219  220
  221  222  223  224  225  226  227  228  229  230  231  232  233  234
  235  236  237  238  239  240  241  242  243  244  245  246  247  248

NameError: name 'get_uci_har_features' is not defined

In [91]:
signals_train, labels_train = randomize(x_train, y_train)
signals_test, labels_test = randomize(x_test, y_test)

In [121]:
dict_classifiers = {
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Linear SVM": SVC(),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB(),   
    "AdaBoost": AdaBoostClassifier(),
    "Gaussian Process": GaussianProcessClassifier()
}
   
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 10, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start =time.perf_counter()
        classifier.fit(X_train, Y_train)
        t_end = time.perf_counter()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models

def get_train_test(df, y_col, x_cols, ratio):
    """ 
    This method transforms a dataframe into a train and test set, for this you need to specify:
    1. the ratio train : test (usually 0.7)
    2. the column with the Y_values
    """
    mask = np.random.rand(len(df)) < ratio
    df_train = df[mask]
    df_test = df[~mask]
       
    Y_train = df_train[y_col].values
    Y_test = df_test[y_col].values
    X_train = df_train[x_cols].values
    X_test = df_test[x_cols].values
    return df_train, df_test, X_train, Y_train, X_test, Y_test
    
def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))
    
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

def get_uci_har_features(dataset, labels, waveletname):
    uci_har_features = []
    for signal_no in range(0, len(dataset)):
        features = []
        for signal_comp in range(0,dataset.shape[2]):
            signal = dataset[signal_no, :, signal_comp]
            list_coeff = pywt.wavedec(signal, waveletname)
            for coeff in list_coeff:
                features += get_features(coeff)
        uci_har_features.append(features)
    X = np.array(uci_har_features)
    Y = np.array(labels)
    return X, Y

In [122]:
waveletname = 'db8'
X_train, Y_train = get_uci_har_features(signals_train, labels_train, waveletname)
X_test, Y_test = get_uci_har_features(signals_test, labels_test, waveletname)

In [123]:
models = batch_classify(X_train, Y_train, X_test, Y_test)
display_dict_models(models)

trained Gradient Boosting Classifier in 2.64 s
trained Random Forest in 0.20 s
trained Logistic Regression in 0.03 s
trained Nearest Neighbors in 0.00 s
trained Decision Tree in 0.02 s


C:\Users\PANKAJ\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


trained Linear SVM in 0.02 s


C:\Users\PANKAJ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


trained Neural Net in 1.25 s
trained Naive Bayes in 0.00 s
trained AdaBoost in 0.02 s
trained Gaussian Process in 0.66 s


,classifier,train_score,test_score,train_time
0,Gradient Boosting Classifier,1.000000,1.000000,2.639071
1,Random Forest,1.000000,1.000000,0.198640
2,Logistic Regression,1.000000,1.000000,0.026962
4,Decision Tree,1.000000,1.000000,0.022552
6,Neural Net,1.000000,1.000000,1.252338
7,Naive Bayes,1.000000,1.000000,0.004386
8,AdaBoost,1.000000,1.000000,0.022275
3,Nearest Neighbors,0.995274,0.997797,0.000507
5,Linear SVM,0.996219,0.997797,0.019395
9,Gaussian Process,1.000000,0.991189,0.664264
